In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output
from sklearn.metrics import (
    precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix
)
from openpyxl import Workbook, load_workbook
import os

# Function to calculate classification metrics
def calculate_classification_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    gini = 2 * auc - 1
    confusion = confusion_matrix(y_true, y_pred)

    metrics = {
        'Metric': ['Precision', 'Recall', 'F1-Score', 'AUC ROC Score', 'GINI'],
        'Value': [precision, recall, f1, auc, gini]
    }
    return metrics, confusion

# Function to save classification results to Excel
def save_classification_results_to_excel(directory, model_name, sheet_name, results, confusion):
    file_name = os.path.join(directory, f'{model_name}_classification_results.xlsx')
    try:
        workbook = load_workbook(file_name)
    except FileNotFoundError:
        workbook = Workbook()
        if 'Sheet' in workbook.sheetnames:
            del workbook['Sheet']

    if sheet_name not in workbook.sheetnames:
        workbook.create_sheet(title=sheet_name)

    sheet = workbook[sheet_name]
    sheet.append(['Metric', 'Value'])

    for metric, value in zip(results['Metric'], results['Value']):
        sheet.append([metric, value])

    sheet.append([])
    sheet.append(['Confusion Matrix'])
    confusion_df = pd.DataFrame(confusion, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1'])
    header = [''] + confusion_df.columns.tolist()
    sheet.append(header)
    for idx, row in confusion_df.iterrows():
        sheet.append([idx] + row.tolist())

    workbook.save(file_name)
    print(f'Classification results saved to {file_name}, sheet: {sheet_name}')

# Main function to run classification metrics
def run_classification_metrics():
    output = widgets.Output()
    results_area = widgets.Output()

    directory_selector = widgets.Text(description='Directory Path:')
    dir_confirm_button = widgets.Button(description='Confirm Directory')
    file_selector_train = widgets.Dropdown(description='sampled_y_train')
    file_selector_test = widgets.Dropdown(description='y_test')
    confirm_files_button = widgets.Button(description='Confirm Files')

    model_name_widget = widgets.Text(description='Model Name:')
    dataset_type_widget = widgets.Dropdown(
        options=[('Select One', 'select_one'), ('Train', 'train'), ('Test', 'test')],
        description='Dataset Type:'
    )

    threshold_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='Threshold:')
    run_button = widgets.Button(description='Run Metrics')

    file_chooser_pred = FileChooser(os.getcwd())
    file_chooser_pred.title = '<b>Select y_pred file</b>'
    file_chooser_pred.filter_pattern = '*.csv'

    selected_directory = {'path': None}
    selected_files = {'train': None, 'test': None}

    def update_file_selectors():
        if selected_directory['path']:
            files = [f for f in os.listdir(selected_directory['path']) if f.endswith('.csv')]
            file_selector_train.options = files
            file_selector_test.options = files
            if files:
                file_selector_train.value = files[0]
                file_selector_test.value = files[0]

    def on_dir_confirm_clicked(b):
        selected_directory['path'] = directory_selector.value.strip()
        if os.path.isdir(selected_directory['path']):
            update_file_selectors()
            with output:
                clear_output()
                display(file_selector_train, file_selector_test, confirm_files_button)
        else:
            with output:
                clear_output()
                print("Invalid directory. Please enter a valid path.")

    def on_confirm_files_clicked(b):
        selected_files['train'] = os.path.join(selected_directory['path'], file_selector_train.value)
        selected_files['test'] = os.path.join(selected_directory['path'], file_selector_test.value)
        with output:
            clear_output()
            display(file_selector_train, file_selector_test, confirm_files_button)
            display(model_name_widget, dataset_type_widget, threshold_slider, file_chooser_pred, run_button)

    def on_run_button_clicked(b):
        with results_area:
            model_name = model_name_widget.value.strip()
            dataset_type = dataset_type_widget.value
            threshold = threshold_slider.value
            file_path = file_chooser_pred.selected

            clear_output(wait=False)

            if not model_name or dataset_type == 'select_one' or not file_path:
                print("Please enter a model name, select a dataset, and choose a prediction file.")
                return

            try:
                y_pred = pd.read_csv(file_path).iloc[:, 0]
            except Exception as e:
                print(f"Error reading prediction file: {e}")
                return

            y_actual_file = selected_files['train'] if dataset_type == 'train' else selected_files['test']
            try:
                y_actual = pd.read_csv(y_actual_file).iloc[:, 0]
            except Exception as e:
                print(f"Error reading actual labels file: {e}")
                return

            y_pred = (y_pred >= threshold).astype(int)

            results, confusion = calculate_classification_metrics(y_actual, y_pred)

            display(widgets.HTML(f'<h3>Model: {model_name}, Dataset: {dataset_type}, Threshold: {threshold:.2f}</h3>'))
            display(pd.DataFrame(results))
            display(pd.DataFrame(confusion, columns=['Predicted 0', 'Predicted 1'], index=['Actual 0', 'Actual 1']))

            save_classification_results_to_excel(
                selected_directory['path'], model_name,
                os.path.basename(file_path), results, confusion
            )

    dir_confirm_button.on_click(on_dir_confirm_clicked)
    confirm_files_button.on_click(on_confirm_files_clicked)
    run_button.on_click(on_run_button_clicked)

    display(widgets.VBox([
        directory_selector,
        dir_confirm_button,
        output,
        results_area
    ]))
